# Generador de Recetas



In [508]:
# import numpy as np
%pylab
import pandas as pd
import networkx as nx


import itertools
from collections import OrderedDict
from collections import Counter
from networkx.algorithms import approximation as apxa
from sklearn import cluster
import random
import numpy as np
# from networkx.algorithms.community import k_clique_communities
from collections import defaultdict
from matplotlib import colors

rcParams.update({'font.size': 23})
rc("text", usetex = True)
rc("font", family = "serif")
rcParams['figure.subplot.hspace'] = 0.3
rcParams['figure.subplot.wspace'] = 0.1
mpl.rcParams['lines.linewidth'] = 4

sns.set(style='whitegrid', palette='muted',rc={'figure.figsize': (8,6)})
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### La primera parte consiste en limpiar los datos recibidos. 

El dataset recibido contiene numerosos detalles que pueden causar un problemas ne el código:

    1. Filas con títulos y Categoría vacía: Estos datos son inconsistentes y van a ser eliminados
    2. Columnas comenzando por '_' o finalizando en 'asterisco' y 'condición': contienen datos irrelevantes
    3. Las columnas de ingredientes , pasos, entre otras , repinten información después del caracter '|'




In [567]:
#If columns matches with the ending of x, eliminates the repeated patterns in x. 
#    Returns df. with a new column having the cleaned data 
def EliminateRepeated(df, x, column):
    if x.endswith(column) :
            #Separate strings by '|'.
            vecs = df[x].str.split(pat = "|",expand=True)  #vecs is the number of divisions of the df[x]
            num = x.split('_') ; num = num[1]  #get number of column
            df[column.upper()+'_'+ num] = vecs[0]  #create new column with the ingredient
            #Add another ingredient to the columns if there is a diferent value in one of them 
            if 1 in vecs:
                condition = ((vecs[0].notnull()) 
                            & (vecs[1].notnull())
                            & (vecs[0] != vecs[1]) )
                #if the new column provides different results, add the next column with a comma between the values            
                df.loc[condition, column.upper()+'_'+ num ] = (vecs.loc[condition, 0]  
                                                              + ','
                                                              + vecs.loc[condition, 1]) 
            del df[x] #finish eliminating df[x], now that the information have been saved in df[x.upper()+'_'+ num] 
    return df      


def cleanData(df): 
    df =  df.loc[df.Categories == 'Recetas']
    df = df[df.Title.notnull()]
    
    del df['Content']
    del df['Excerpt']
    del df['URL']
    
    for x in df.columns:
        #eliminate vacum columns
        if df[x].count() < 1:
            del df[x]
        #eliminate _ingredient columns. They have useless information    
#         elif x.startswith('_ingredient'):
        elif x.startswith('_'):
            del df[x]
            
        #eliminate aterisco columns. They have useless information     
#         elif x.endswith('asterisco'):
#             del df[x]
#         elif x.endswith('condicion'):
#             del df[x]

    for x in df.columns:
        #Eliminate repeated terms in 'ingredientes', 'cantidades' and 'pasos'
        EliminateRepeated(df, x, 'ingrediente')
        EliminateRepeated(df, x, 'cantidad')
        EliminateRepeated(df, x, 'paso')
    return df       

#Fills null values with  empty strings
#FIND POSSIBLE MISLEADING VALUES ON THE DATA SET AND REPLACE THEM BY NOTHING
def replaceValues(df, x):
    df[x] = df[x].fillna(value = '')
    df[x] = df[x].str.replace('®' , '' )
    df[x] = df[x].str.replace('<sup>®</sup>' , '' )
    df[x] = df[x].str.replace('<sup></sup>' , '' )
    df[x] = df[x].str.replace('<sup>®</sup>' , '' )
    df[x] = df[x].str.replace('®' , '' )
    df[x] = df[x].str.replace('<strong>' , '' )
    df[x] = df[x].str.replace('</strong>' , ':' )
    df[x] = df[x].str.replace('<b>' , '' )
    df[x] = df[x].str.replace('</b>' , '' )
    df[x] = df[x].str.replace('<i>' , '' )
    df[x] = df[x].str.replace('</i>' , '' )
    df[x] = df[x].str.replace(')' , '' )
    df[x] = df[x].str.replace('(' , '' )
    df[x] = df[x].str.replace('?' , '' )
    df[x] = df[x].str.replace('¿' , '' )
    df[x] = df[x].str.replace('¡' , '' )
    df[x] = df[x].str.replace('!' , '' )
    df[x] = df[x].str.replace('/' , '' )
    return df[x]
df = pd.read_csv('Recetas.csv', delimiter = ';')
df[{'Title','Categories'}].loc[df.Categories == 'Recetas'].count()
df = cleanData(df)

Gracias a una limpieza efectiva del archivo, hemos encontrado que los datos más relevantes se encuentran en las columnas 
1. Title
2. Tags
3. Ingredientes
4. Pasos
5. Cantidad


In [565]:
# 
def cleanCVS(cvs_file):
    df = pd.read_csv(cvs_file, delimiter = ';')
    df[{'Title','Categories'}].loc[df.Categories == 'Recetas'].count()
    df = cleanData(df)

    s = ['Title', 'Tags']
    df['Tags'] = df['Tags'].str.replace('|' , ' , ' )

    # x = 'INGREDIENTE_ingrediente'
    # df[x] = replaceValues(df, x)
    # df['Total_Ingredientes'] = df['INGREDIENTE_ingrediente']
    # del df['INGREDIENTE_ingrediente']

    for x in df.columns:
        if x.startswith('INGREDIENTE') :
            df[x] = replaceValues(df, x)
            s.append(x)


    #         df['Total_Ingredientes']  = df['Total_Ingredientes'] + ' , ' + df[x]
    #         del df[x]
    # df['Total_Ingredientes'] = df['Total_Ingredientes'].str.replace(',,' , '' )


#     df['PASOS'] = ""

    del df['PASO_nombre']
    del df['PASO_descripcion']


    for x in df.columns:
         if x.startswith('PASO') :
            df[x] = replaceValues(df, x)
            s.append(x)
         
    #             df['PASOS'] = df['PASOS'] + '.' + df[x]
#     s.append('PASOS')        

    #         s.append(x)
    # new = df['PASOS'].str.split(pat = ".",expand=True)          

    for x in df.columns:
         if x.startswith('CANTIDAD') :
            df[x] = replaceValues(df, x)
            s.append(x)



    data = df[s].copy()

    for x in data.columns:
        if x != 'Title':
            data['Title']  = data['Title'] + ' , ' + data[x]
            del data[x]

    df[s].T
    df[s].to_csv('CleanedData.csv')
    return df[s]
df= cleanCVS('Recetas.csv')

In [556]:
display_all(df.describe(include='all').T)

,count,unique,top,freq
Title,96,95,Torta invertida de piña y queso crema,2
Tags,96,96,"Bebida , Cóctel , Cremosa , Deliciosa , Piña c...",1
INGREDIENTE_0,96,71,,21
INGREDIENTE_1,96,63,,22
INGREDIENTE_2,96,68,,23
INGREDIENTE_3,96,62,,26
INGREDIENTE_4,96,48,,45
INGREDIENTE_5,96,42,,51
INGREDIENTE_6,96,36,,58
INGREDIENTE_7,96,27,,68


# Análisis
En el archivo 'CleanedData.csv' se encuentran las  recetas en una forma mucho más fácil de leer. Analizando ester archivo encontramos los siguientes hallazgos. 
    1. Los pasos a seguir se describen en las columnas 
    2. 

In [566]:
df

,Title,Tags,INGREDIENTE_0,INGREDIENTE_1,INGREDIENTE_2,INGREDIENTE_3,INGREDIENTE_4,INGREDIENTE_5,INGREDIENTE_6,INGREDIENTE_7,...,CANTIDAD_6,CANTIDAD_7,CANTIDAD_8,CANTIDAD_9,CANTIDAD_10,CANTIDAD_11,CANTIDAD_12,CANTIDAD_13,CANTIDAD_14,CANTIDAD_cantidad
0,Pollo Cancún,"Aceite de oliva , Cena , Cilantro , Fruta trop...","DOLE Pineapple Slices piña en rebanadas, sin e...",De pollo deshuesadas y sin piel,Comino molido,Orégano seco desmenuzado,Clavo molido,Aceite de oliva,DOLE Pineapple Juice jugo de piña,Jugo de limón,...,34 taza,1 cda.,1 cdta.,,,,,,,
3,Paletas heladas caribeñas de crema,"Fruta tropical , Mango , Miel , Papaya , Postr...",Jarred DOLE Tropical Fruit fruta tropical enva...,Néctar de mango o papaya,Leche de coco light baja en calorías,Plátano DOLE pelado y rebanado,Queso crema suavizado,Miel,Moldes para paletas heladas o vasitos de plást...,,...,10,,,,,,,,,
6,Pizza de carnitas y piña,"Cena , Cerdo , Chipotle , Cilantro , Queso",,,,,,,,,...,,,,,,,,,,
7,Pescado a la veracruzana,"Aceite de oliva , Arroz , Cena , Cilantro , Ja...",,,,,,,,,...,,,,,,,,,,
10,Lomo con chimichurri a la parrilla,"Aceite de oliva , Cena , Cilantro , Lomo , Res...",Hojas frescas de perejil italiano,Aceite de oliva,Vinagre de vino rojo,Hojas de cilantro fresco,,,,,...,,,,,,,,,,
11,Fajitas de pollo y frijoles negros,"Cena , Frijoles , Pollo , Queso , Salsa , Tort...",,,,,,,,,...,,,,,,,,,,
14,Rollo de galletas brazo gitano,"Fruta tropical , Postre",Galletas redondas de vainilla,Jugo DOLE Pineapple piña,Leche condensada,Nueces,Nueces pecanas,Arándanos,DOLE Tropical Fruit fruta tropical o Mixed Fru...,,...,1 taza,,,,,,,,,
17,Guacamole con fruta tropical,"Aguacate , Guacamole","Aguacates o paltas machacados, sin semilla ni ...",Tomates cortados en cubos pequeños,Cebolla blanca finamente picada,"DOLE Pineapple Tidbits solo la piña, sin el jugo",Jugo de limón,Sal y pimienta,,,...,,,,,,,,,,"2,14 taza,12 taza,1 taza 8 oz,2 cdas.,"
18,Biónico,"Frutas , Miel , Postre","DOLE Pineapple Chunks piña en trozos, sin el jugo","DOLE Mandarin Oranges mandarinas, sin el jugo","DOLE Jarred Peaches duraznos envasados, sin el...",Leche condensada,Crema agria,Miel,Canela,Granola,...,1 cdta.,13 taza,13 taza,,,,,,,
21,Ceviche de camarón,"Ceviche , Cilantro , Mariscos","Camarones pequeños, cocidos","DOLE Pineapple Chunks piña en trozos, sin el ...","Tomates pequeños, cortados finamente","Cebolla, picada finamente","Hojas de cilantro, corte chiffonade",Jugo de limón,Sal y pimienta,,...,,,,,,,,,,


In [561]:
df['PASOS'] = ""

if x.startswith('PASO') :
    df['PASOS'] = df['PASOS'] + '.' + df[x]
    # s.append('PASOS')        
    s.append(x)
# pasos = df['PASOS'].str.split(pat = ".")
df


In [562]:
pasos

0      []
3      []
6      []
7      []
10     []
11     []
14     []
17     []
18     []
21     []
22     []
23     []
26     []
29     []
30     []
31     []
32     []
33     []
36     []
37     []
40     []
43     []
44     []
45     []
46     []
47     []
48     []
49     []
50     []
51     []
       ..
88     []
89     []
90     []
91     []
92     []
93     []
94     []
95     []
96     []
97     []
98     []
99     []
100    []
101    []
102    []
103    []
104    []
105    []
106    []
107    []
108    []
109    []
110    []
111    []
112    []
113    []
114    []
115    []
116    []
117    []
Name: PASOS, Length: 96, dtype: object

In [553]:
r(12)

'12'

In [485]:
data

,Title
0,"Pollo Cancún , Aceite de oliva , Cena , Cilant..."
3,"Paletas heladas caribeñas de crema , Fruta tro..."
6,"Pizza de carnitas y piña , Cena , Cerdo , Chip..."
7,"Pescado a la veracruzana , Aceite de oliva , A..."
10,"Lomo con chimichurri a la parrilla , Aceite de..."
11,"Fajitas de pollo y frijoles negros , Cena , Fr..."
14,"Rollo de galletas brazo gitano , Fruta tropica..."
17,"Guacamole con fruta tropical , Aguacate , Guac..."
18,"Biónico , Frutas , Miel , Postre , , DOLE Pi..."
21,"Ceviche de camarón , Ceviche , Cilantro , Mari..."


In [486]:
data.head()

,Title
0,"Pollo Cancún , Aceite de oliva , Cena , Cilant..."
3,"Paletas heladas caribeñas de crema , Fruta tro..."
6,"Pizza de carnitas y piña , Cena , Cerdo , Chip..."
7,"Pescado a la veracruzana , Aceite de oliva , A..."
10,"Lomo con chimichurri a la parrilla , Aceite de..."


In [487]:
[]

[]

In [488]:
def filterWords(data):
    words = array(data['Title'])
    Recetas = []
    # These words are going to be eliminated from the list
    delated = [ ''
               ,'de'
               ,'y'
               ,'la'
               ,'el'
               ,'en'
               ,'una'
               ,'con'
               ,'los'
               ,'un'
               ,'hasta'
               ,'para'
               ,'a'
               ,'que'
               ,'o'
               ,'las'
               ,'los'
               ,'por'
               ,'al'
               ,'del'
               ,'esté'
               ,'sobre'
               ,'1'
               ,'2'
               ,'3'
               ,'se'
               ,'durante'
               ,'sin'
               ,'coloca'
               ,'minutos']
    for w in words:
        st = w.split(',')
        l = []
        for s in st:
            x = s.split(' ')
            l = l + x
        l = [x.lower().strip() for x in l] #strip each word
        l = list(filter(lambda a: a not in delated, l)) # clean misleading words
        l = list(OrderedDict.fromkeys(l))  #Eliminate repeated words in a row
        Recetas.append(l)       #fd

    for r in Recetas:
        for j in range(len(r)): 
            if 'dole' in r[j]:
                r[j] = 'dole'
    return Recetas
    # eliminating next  part whe attempting the final model           
    # Recetas = Recetas[:]

Recetas = filterWords(data) 

Words = list(itertools.chain.from_iterable(Recetas))
Word_labels = list(OrderedDict.fromkeys(Words))
# W = pd.DataFrame(Words)


# W[0].value_counts()

# Word_labels
c = Counter(Words)
h = c.most_common(60)
h = array(h)
Word_labels = h[:,0]

In [489]:
# Word_labels
# Recetas[0]
Word_labels

array(['dole', 'piña', 'jugo', 'pineapple', 'azúcar', 'sal', 'trozos',
       'piñas', 'postre', 'limón', 'harina', 'bebida', 'leche', 'canela',
       'dulce', 'tropical', 'salsa', 'mantequilla', 'rodajas', 'agua',
       'coco', 'vainilla', 'trituradas', 'polvo', 'aceite', 'cena',
       'fruta', 'crema', 'queso', 'fácil', 'escurridas', 'cilantro',
       'juice', 'condensada', 'naranja', 'delicioso', 'méxico',
       'extracto', 'común', 'lima', 'molida', 'crushed', 'cerdo',
       'cebolla', 'ajo', 'verde', 'maíz', 'desayuno', 'huevos', 'blanco',
       'alcohólica', 'pollo', 'finamente', 'tidbits', 'pimienta',
       'chunks', 'ron', 'cóctel', 'sabor', 'pastel'], dtype='<U11')

In [490]:
G = nx.Graph()
G.add_nodes_from(array(Word_labels))
Recetas_copy = Recetas.copy()
elist = []
nodeSize = zeros(len(Word_labels))

alpha = 1/len(Recetas)
# 'piña' in m
for i in range(len(Word_labels)):
    w = Word_labels[i]
    for j in arange(i+1 , len(Word_labels)):
        w2 = Word_labels[j]
#         print(w,w2)
        count = 0
        for R in Recetas:
            if (w in R) & (w2 in R):
                count = count +1
                nodeSize[i] = nodeSize[i] + 1
                nodeSize[j] = nodeSize[j] + 1
#                 print(count)
        if count >2:
            elist.append((w,w2,count*alpha))
G.add_weighted_edges_from(elist)
    
    
#     for R in Recetas_copy:
#         if (w in R):
#             for w2 in R:
#                 if (w2 in Word_labels):
#                     G.add_edge(w, w2)
#         found = False
#         while (found = False)
        
#             if 
eds = array(elist)
#Filter nodes coupled with dole    
G2 = nx.ego_graph(G, 'dole')

#The node size will be greater when the node is connected with more words
nodes = list(G2.nodes())
nodeSizes = zeros(len(nodes))
for i in range(len(nodes)):
    x = nodes[i]
    nodeSizes[i] = len([n for n in G2.neighbors(x)])
    
# nodeSizes
mini = min(eds[:,2])
maxi = max(eds[:,2])
edges,weights = zip(*nx.get_edge_attributes(G2,'weight').items())
# print(mini,maxi)
# eds[:,2]
# graphs = list(nx.connected_component_subgraphs(G))
# G2 = graphs[0]

In [491]:
print(G2.number_of_nodes())
nx.draw_networkx( G2 , pos=nx.spring_layout(G2)
                 , node_size = nodeSize
                 , with_labels = True
                 , node_color= 'r'
                 , edgelist = edges
                 , width= 1.0
                 , edge_color = weights
                 , edge_cmap = cm.Blues
                 , edge_vmin = 0 
                 , edge_vmax = 0.6)
# print(nx.triangles(G,'piña'))

60


/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [414]:
print(random.randrange(10, 50, 5))

45


In [418]:
y_true = np.random.randint(2, size=len(nodes))

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0])

In [421]:
nx.draw_communities(G2, y_true, pos)

AttributeError: module 'networkx' has no attribute 'draw_communities'

In [440]:
# G = nx.complete_graph(5)
# K5 = nx.convert_node_labels_to_integers(G,first_label=2)
# G.add_edges_from(K5.edges())
c = list(k_clique_communities(G2, 4))
# print(list(c[0]))
# print(list(k_clique_communities(G2, 5)))
list(k_clique_communities(G2, 2))

[frozenset({'aceite',
            'agua',
            'ajo',
            'alcohólica',
            'azúcar',
            'bebida',
            'blanco',
            'canela',
            'cebolla',
            'cena',
            'cerdo',
            'chunks',
            'cilantro',
            'coco',
            'común',
            'condensada',
            'crema',
            'crushed',
            'cóctel',
            'delicioso',
            'desayuno',
            'dole',
            'dulce',
            'escurridas',
            'extracto',
            'finamente',
            'fruta',
            'fácil',
            'harina',
            'huevos',
            'jugo',
            'juice',
            'leche',
            'lima',
            'limón',
            'mantequilla',
            'maíz',
            'molida',
            'méxico',
            'naranja',
            'pastel',
            'pimienta',
            'pineapple',
            'piña',
            'piñas',
  

In [473]:
def graph_to_edge_matrix(G):
    """Convert a networkx graph into an edge matrix.
    See https://www.wikiwand.com/en/Incidence_matrix for a good explanation on edge matrices
   
    Parameters
    ----------
    G : networkx graph
    """
    G = nx.convert_node_labels_to_integers(G,first_label=0)
    # Initialize edge matrix with zeros
    edge_mat = np.zeros((len(G), len(G)), dtype=int)

    # Loop to set 0 or 1 (diagonal elements are set to 1)
    for node in G:
        for neighbor in G.neighbors(node):
            edge_mat[node][neighbor] = 1
#             print(node,neighbor)
        edge_mat[node][node] = 1

    return edge_mat
edge_mat = graph_to_edge_matrix(G2)

In [494]:
algorithm = cluster.AffinityPropagation(damping=0.6)
algorithm.fit(edge_mat)
# Fit all models
for e,c in enumerate(algorithm.labels_):
     print(e,c)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 3
8 2
9 6
10 3
11 1
12 2
13 2
14 2
15 1
16 4
17 2
18 6
19 3
20 2
21 2
22 3
23 2
24 3
25 3
26 2
27 2
28 6
29 6
30 3
31 4
32 1
33 2
34 5
35 5
36 1
37 2
38 3
39 1
40 3
41 3
42 4
43 4
44 4
45 1
46 5
47 3
48 2
49 1
50 1
51 5
52 4
53 4
54 4
55 5
56 5
57 5
58 6
59 6


In [502]:
def draw_communities(G, membership):
    """Draws the nodes to a plot with assigned colors for each individual cluster
    Parameters
    ----------
    G : networkx graph
    membership : list
        A list where the position is the student and the value at the position is the student club membership.
        E.g. `print(membership[8]) --> 1` means that student #8 is a member of club 1.
    pos : positioning as a networkx spring layout
        E.g. nx.spring_layout(G)
    """ 
    fig, ax = subplots(figsize=(16,9))
    
    # Convert membership list to a dict where key=club, value=list of students in club
    club_dict = defaultdict(list)
    for student, club in enumerate(membership):
        club_dict[club].append(student)
    
    # Normalize number of clubs for choosing a color
    norm = colors.Normalize(vmin=0, vmax=len(club_dict.keys()))
    
    edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    for club, members in club_dict.items():
        nx.draw_networkx( G , pos=nx.spring_layout(G)
                 , node_size = nodeSize
                 , with_labels = True
                 , nodelist=members
                 , node_color=cm.jet(norm(club))
                 , edgelist = edges
                 , width= 1.0
                 , edge_color = weights
                 , edge_cmap = cm.Blues
                 , edge_vmin = 0 
                 , edge_vmax = 0.6)

    # Draw edges (social connections) and show final plot
    title("Recepies")
#     nx.draw_networkx_edges(G, pos, alpha=0.5, ax=ax)

In [505]:
draw_communities(G2,algorithm.labels_ )

NetworkXError: Node 0 has no position.

In [477]:
G=nx.path_graph(4)
pos=nx.spring_layout(G)
nx.draw_networkx( G, pos)

/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [459]:
nx.draw_networkx( G2)

/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [ ]:
l = [1,2,4,1,3,4,6,7,8,1,4]
list(filter(lambda a: a not in [1 , 4, 8], l)) 

In [111]:
dolead = G.adjacency()
# G.add_node(1)
# G.add_node('b')
G.subgraph(dolead['dole'])

TypeError: 'dict_itemiterator' object is not subscriptable

In [101]:
k_components = apxa.k_components(G)

In [103]:
k_components[1]

[{'3',
  '<i>light<i>',
  'aceite',
  'aguacate',
  'arroz',
  'arándanos',
  'baja',
  'brazo',
  'calorías',
  'cancún',
  'caribeñas',
  'carnitas',
  'cebolla',
  'cena',
  'cerdo',
  'chimichurri',
  'chipotle',
  'chunks',
  'cilantro',
  'clavo',
  'coco',
  'comino',
  'condensada',
  'cortados',
  'crema',
  'deshuesadas',
  'desmenuzado',
  'dole',
  'envasada',
  'fajitas',
  'finamente',
  'frescas',
  'fresco',
  'frijoles',
  'fruit',
  'fruta',
  'galletas',
  'gitano',
  'guacamole',
  'heladas',
  'hojas',
  'italiano',
  'jalapeño',
  'jarred',
  'jugo',
  'juice',
  'leche',
  'limón',
  'lomo',
  'mango',
  'miel',
  'mixed',
  'moldes',
  'molido',
  'negros',
  'nueces',
  'néctar',
  'oliva',
  'orégano',
  'oz',
  'paletas',
  'papaya',
  'parrilla',
  'pecanas',
  'pelado',
  'pequeños',
  'perejil',
  'pescado',
  'picada',
  'piel',
  'pimienta',
  'pineapple',
  'pizza',
  'piña',
  'plástico',
  'plátano',
  'pollo',
  'postre',
  'queso',
  'rebanadas',
  

In [96]:
G = nx.petersen_graph()
k_components = apxa.k_components(G)

In [97]:
k_components

defaultdict(list,
            {1: [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}],
             2: [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}],
             3: [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}]})

In [98]:
nx.draw_networkx(G)

/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [ ]:
df['Ingredientes_ingrediente'].str.split(pat = "|",expand = True)

In [ ]:
display_all(df.describe(include='all').T)

In [ ]:
df['tags'] = df['Tags'].str.split(pat = "|")

In [ ]:
df[{'Title','tags'}]

In [ ]:
df.head()

In [ ]:
df['ingredientes_11_asterisco'].value_counts()

In [ ]:
s = {'Ingredientes_ingrediente', 'Ingredientes_cantidad' , 'Ingredientes_nombre_de_condicion'}
df[s]

In [ ]:
x = 'ingredientes_9_ingrediente'
ingredients = df[x].str.split(pat = "|",expand=True)
# ingredients[(ingredients[0].notnull()) & (ingredients[0] != ingredients[1]) & (ingredients[1].notnull()) ]
# ingredients = df.ingredientes_17_ingrediente.notnull()
# ingredients[df.ingredientes_17_ingrediente.notnull()]
# 1 in ingredients
num = x.split('_')
num = num[1]
print(num)
df['Ingrediente_'+ num] = ingredients[0] 

if 1 in ingredients:
                print('entered')
                condition = ((ingredients[0].notnull()) 
                           & (ingredients[0] != ingredients[1]) 
                           & (ingredients[1].notnull()))
                    
                
#                 df['Ingrediente_'+ num] = ingredients[0] +','+ ext[1]

df.loc[condition, 'Ingrediente_'+ num ] = (ingredients.loc[condition, 0]  + ','+ ingredients.loc[condition, 1] ) 
condition = ((ingredients[0].notnull()) 
                           & (ingredients[0] != ingredients[1]) 
                           & (ingredients[1].notnull()))

# df['Ingrediente_'+ num]
# ext
df.loc[condition,'Ingrediente_'+ num]

In [ ]:
display_all(df.describe(include='all').T)

In [ ]:
df['Nombre del Autor'].value_counts()

In [ ]:
# for x in df.columns:
#     if df[x].count() < 10:
#         del df[x]
df.head()

In [ ]:
df['Categories'].value_counts()

In [ ]:
df['ingredientes_0_cantidad'].value_counts()

In [ ]:
df['_ingredientes_0_cantidad'].value_counts()

In [ ]:
df['Content'].value_counts()